In [15]:
import json

In [3]:
inputFile = "./allSeps.jsonl"

In [4]:
allKeys = set()
languages = set()
allDays = set()

In [5]:
# Read and find keys
print("Reading File...")
with open(inputFile, encoding='utf-16') as f:
    # next(f)
    for num, line in enumerate(f, 1):
        if num % 1000 == 0:
            print(f"Reading line {num}...")

        j = json.loads(line)
        # Append keys
        allKeys.update(j.keys())

        # Append language values
        languages.add(j["steam.supported_languages"])

        # Append all the timestamps
        sales = j['itad.sales']

        allDays.update([row[0] for row in sales])

Reading File...
Reading line 1000...
Reading line 2000...
Reading line 3000...
Reading line 4000...
Reading line 5000...
Reading line 6000...
Reading line 7000...
Reading line 8000...
Reading line 9000...
Reading line 10000...
Reading line 11000...
Reading line 12000...
Reading line 13000...
Reading line 14000...
Reading line 15000...
Reading line 16000...
Reading line 17000...
Reading line 18000...
Reading line 19000...
Reading line 20000...
Reading line 21000...
Reading line 22000...
Reading line 23000...
Reading line 24000...
Reading line 25000...
Reading line 26000...
Reading line 27000...
Reading line 28000...
Reading line 29000...
Reading line 30000...
Reading line 31000...
Reading line 32000...
Reading line 33000...
Reading line 34000...
Reading line 35000...


In [6]:
# Output files
print('Writing Files...')
with open("keys.txt", 'w') as f:
    for key in sorted(allKeys):
        f.write(f"{key}\n")

with open("days.txt", 'w') as f:
    for day in sorted(allDays):
        f.write(f"{day}\n")

with open("languages.txt", 'w') as f:
    for langStr in sorted(languages):
        f.write(f"{langStr}\n")
print("Done!")

Writing Files...
Done!


In [7]:
# Manually find a definitive list of languages that the games support (for 1hot encoding)
# We consider all spanish the same (spain, brazil, latinamerica)
SUPPORTED_LANGUAGES = ['english', 'french', 'german', 'dutch', 'czech', 'hungarian', 'italian', 'japanese', 'polish', 'bulgarian', 'danish', 'spanish', 'finnish', 'greek', 'korean', 'norwegian', 'portuguese', 'romanian', 'russian', 'simplified chinese', 'traditional chinese', 'swedish', 'thai', 'turkish', 'ukrainian', 'vietnamese', 'arabic']

buf  = []
with open("languages.txt", 'r') as f:
    for line in f:
        line.replace(" ", '')
        for lang in SUPPORTED_LANGUAGES:
            line = line.replace(lang, "")
        line = line.replace("  ", ' ').replace('\n', '').replace('spain', '').replace('brazil', '').replace(' ', '').replace('latinamerica', '').replace('only', '')
        if len(line) != 0:
            buf.append(line)
buf

[]

In [8]:
# Find the interval between each time period (let's hope it's constant, so we can use normal forecasting methods)

differences = {}
with open('days.txt', 'r') as f:
    prev = next(f)
    for line in f:
        difference = int(line) - int(prev)
        if difference in differences:
            differences[difference] += 1
        else:
            differences[difference] = 1
        prev = line

differences

# We see that the differences are very irregular

{2660183000: 1,
 842000: 1,
 603631000: 1,
 1263000: 1,
 6300399000: 1,
 351243000: 1,
 1285000: 1,
 257967000: 1,
 620000: 1,
 499302000: 1,
 1284000: 1,
 16976000: 1,
 9538000: 1,
 41916000: 1,
 4692000: 1,
 298359000: 1,
 6336000: 1,
 54406000: 1,
 533516000: 1,
 4838000: 1,
 2135492000: 1,
 1000: 7,
 294174000: 1,
 1199000: 1,
 25206000: 1,
 1437323000: 1,
 3596000: 1,
 334801000: 1,
 1793000: 1,
 5792414000: 1,
 1794000: 1,
 352833000: 1,
 1776000: 1,
 216969000: 1,
 302296000: 1,
 1243000: 1,
 79655000: 1,
 1293570000: 1,
 1683000: 1,
 521999000: 1,
 1784000: 1,
 1720797000: 1,
 1816000: 1,
 1206050000: 1,
 1745000: 1,
 6919178000: 1,
 1817000: 1,
 336588000: 1,
 1799000: 2,
 7520409000: 1,
 1795000: 1,
 1114258000: 1,
 1737000: 2,
 3704401000: 1,
 1814000: 2,
 347391000: 1,
 5711381000: 1,
 511238000: 1,
 1767000: 1,
 1827008000: 1,
 1785000: 3,
 511204000: 1,
 1815000: 1,
 1974670000: 1,
 1735000: 1,
 1216808000: 1,
 1768000: 1,
 12928275000: 1,
 1573685000: 1,
 1212453000: 1,


In [64]:
from datetime import datetime, timezone
days = set()
times = set()
for timestamp in allDays:
    dt = datetime.fromtimestamp(int(timestamp) / 1000,tz=timezone.utc)
    times.add(dt.time())
    days.add(dt.date())


NameError: name 'allDays' is not defined

In [10]:
# This shows that even in aggregate, many days are missed
# It is prudent in this case to assume a forward fill of the price data.
# E.g. if the price data has:
#   Day 0:  $1
#   Day 10: $2
# We should assume that the price is $1 for days 0-9 (inclusive), then only changes to $2 on the 10th day.
days

{datetime.date(2016, 2, 20),
 datetime.date(2016, 3, 22),
 datetime.date(2016, 3, 29),
 datetime.date(2016, 6, 10),
 datetime.date(2016, 6, 14),
 datetime.date(2016, 6, 17),
 datetime.date(2016, 6, 23),
 datetime.date(2016, 6, 24),
 datetime.date(2016, 6, 27),
 datetime.date(2016, 6, 28),
 datetime.date(2016, 7, 4),
 datetime.date(2016, 7, 29),
 datetime.date(2016, 8, 1),
 datetime.date(2016, 8, 2),
 datetime.date(2016, 8, 18),
 datetime.date(2016, 8, 22),
 datetime.date(2016, 10, 28),
 datetime.date(2016, 11, 1),
 datetime.date(2016, 11, 4),
 datetime.date(2016, 11, 7),
 datetime.date(2016, 11, 8),
 datetime.date(2016, 11, 23),
 datetime.date(2016, 11, 29),
 datetime.date(2016, 12, 19),
 datetime.date(2017, 1, 2),
 datetime.date(2017, 3, 23),
 datetime.date(2017, 3, 27),
 datetime.date(2017, 6, 22),
 datetime.date(2017, 7, 5),
 datetime.date(2017, 8, 17),
 datetime.date(2017, 8, 21),
 datetime.date(2017, 10, 26),
 datetime.date(2017, 11, 1),
 datetime.date(2017, 11, 22),
 datetime.dat

In [63]:
# From this data, make up the following rule:
# Anything before 1AM is probably a late run, and will be considered the price of the previous day
# Anything after that, will be pushed to the end of the day.
#    Our lowest time unit will be days.
#
times

NameError: name 'times' is not defined

In [12]:
# OK! Let's 1-hot encode.
inputFile = 'allSeps.jsonl'
outputFile = 'language1Hot.jsonl'

print("Reading File...")
with open(outputFile, "a") as out:
    with open(inputFile, 'r', encoding='utf-16') as f:
        # next(f)
        for num, line in enumerate(f, 1):
            if num % 1000 == 0:
                print(f"Reading line {num}...")
            j = json.loads(line)
            langStr = j["steam.supported_languages"]

            for lang in SUPPORTED_LANGUAGES:
                attribName = f"steam.supported_languages.{lang.replace(' ', '_')}"
                if lang in langStr:
                    j[attribName] = 1
                else:
                    j[attribName] = 0

            j.pop('steam.supported_languages', None)
            out.write(json.dumps(j) + "\n")

print("done")


Reading File...
Reading line 1000...
Reading line 2000...
Reading line 3000...
Reading line 4000...
Reading line 5000...
Reading line 6000...
Reading line 7000...
Reading line 8000...
Reading line 9000...
Reading line 10000...
Reading line 11000...
Reading line 12000...
Reading line 13000...
Reading line 14000...
Reading line 15000...
Reading line 16000...
Reading line 17000...
Reading line 18000...
Reading line 19000...
Reading line 20000...
Reading line 21000...
Reading line 22000...
Reading line 23000...
Reading line 24000...
Reading line 25000...
Reading line 26000...
Reading line 27000...
Reading line 28000...
Reading line 29000...
Reading line 30000...
Reading line 31000...
Reading line 32000...
Reading line 33000...
Reading line 34000...
Reading line 35000...
done


In [5]:
# Sanity check to make sure we're not including a game many times
inputFile = 'data.json'

freq = {}

with open(inputFile, 'r') as f:
    for num, line in enumerate(f, 1):
        if num % 1000 == 0:
            print(f"Reading line {num}...")
        j = json.loads(line)

        plain = j['itadPlain']
        if plain in freq:
            freq[plain] += 1
        else:
            freq[plain] = 1
freq

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 633: character maps to <undefined>

In [2]:
import json
lines = ''
with open("data.json", encoding="utf-8") as f:
    lines = ''.join(f.readlines())

allData = json.loads(lines)


In [3]:
for game in allData:
    print(game['itadPlain'])

nierautomata
stardewvalley
divinityoriginalsiniidefinitiveedition
portalii
witcheriiiwildhunt
falloutiv
rimworld
terraria
elderscrollsvskyrim
grandtheftautov
leftivdeadii
portal
cyberpunkii0viivii
borderlandsii
tombraider
bioshockinfinite
nomanssky
factorio
cuphead
doom
sidmeierscivilizationv
halflifeii
elderscrollsvskyrimspecialedition
dishonored
deadcells
falloutnewvegas
hollowknight
halflifeiilostcoast
bastion
discoelysium
darksoulsiii
sekiroshadowsdietwice
amnesiadarkdescent
undertale
starbound
rocketleague
limbo
witcheriiassassinsofkingsenhancededition
dishonoredii
ftlfasterthanlight
xcomii
metroii0iiiiii
kerbalspaceprogram
riseoftombraiderii0yearcelebration
subnautica
metalgearsolidvphantompain
reddeadredemptionii
xcomenemyunknown
walkingdead
oriandblindforestdefinitiveedition
blackmesa
stanleyparable
garrysmod
psychonauts
batmanarkhamcitygameofyearedition
papersplease
divinityoriginalsinenhancededition
sidmeierscivilizationvi
darkestdungeon
specopsline
saintsrowthird
crusaderkin

In [4]:
allData[0]

{'steamId': '524220',
 'itadPlain': 'nierautomata',
 'itad.position': 1,
 'itad.rank': 1,
 'itad.sales': [[1456005749000, 19.99],
  [1458665932000, 19.99],
  [1458666774000, 11.99],
  [1459270405000, 11.99],
  [1459271668000, 19.99],
  [1465572067000, 19.99],
  [1465923310000, 19.99],
  [1465924595000, 11.99],
  [1466182562000, 11.99],
  [1466183182000, 19.99],
  [1466682484000, 19.99],
  [1466683768000, 19.99],
  [1466700744000, 19.99],
  [1466710282000, 11.99],
  [1466752198000, 11.99],
  [1466756890000, 11.99],
  [1467055249000, 11.99],
  [1467061585000, 11.99],
  [1467115991000, 11.99],
  [1467649507000, 11.99],
  [1467654345000, 19.99],
  [1469789837000, 19.99],
  [1469789838000, 15.78],
  [1470084012000, 15.78],
  [1470085211000, 19.99],
  [1470110417000, 19.99],
  [1471547740000, 19.99],
  [1471551336000, 10.99],
  [1471886137000, 10.99],
  [1471887930000, 19.99],
  [1477680344000, 19.99],
  [1477682138000, 9.99],
  [1478034971000, 9.99],
  [1478036747000, 19.99],
  [14782537160

In [5]:
allCategoriesSet = set()
allGenresSet = set()

for game in allData:
    allCategoriesSet.update(game['steam.categories_description'])
    allGenresSet.update(game['steam.genres_description'])

allCategories = sorted(allCategoriesSet)
allGenres = sorted(allGenresSet)

In [52]:
from copy import copy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timezone, timedelta, date

analyzer = SentimentIntensityAnalyzer()
# We consider all types of spanish (spain, brazil) the same. This is because there are many ways to phrase it, and it is hard to clean.

SUPPORTED_LANGUAGES = ['english', 'french', 'german', 'dutch', 'czech', 'hungarian', 'italian', 'japanese', 'polish', 'bulgarian', 'danish', 'spanish', 'finnish', 'greek', 'korean', 'norwegian', 'portuguese', 'romanian', 'russian', 'simplified chinese', 'traditional chinese', 'swedish', 'thai', 'turkish', 'ukrainian', 'vietnamese', 'arabic']

def cleanup_data(game):
    """
    Takes a single game object, and 1-hot encodes:
      - Supported Languages
      - Genres description
      - Categories description
    and runs sentiment analysis on:
      - short description
      - detailed description
      - about the game
    :param game:
    :return:
    """
    # Clone the data so we can run this multiple times on the same data
    flattened_game = copy(game)

    # Languages
    lang_str = flattened_game["steam.supported_languages"]
    for language in SUPPORTED_LANGUAGES:
        attrib_name = f"steam.supported_languages.{language.replace(' ', '_')}".lower()
        if language in lang_str:
            flattened_game[attrib_name] = 1
        else:
            flattened_game[attrib_name] = 0
    flattened_game.pop('steam.supported_languages', None)

    # Genres
    for genre in allGenres:
        attrib_name = f"steam.genres.{genre.replace(' ', '_')}".lower()
        if genre in flattened_game['steam.genres_description']:
            flattened_game[attrib_name] = 1
        else:
            flattened_game[attrib_name] = 0
    flattened_game.pop('steam.genres_description', None)

    # Categories
    # 'steam.categories_description'
    for category in allCategories:
        attrib_name = f"steam.categories.{category.replace(' ', '_')}".lower()
        if category in flattened_game['steam.categories_description']:
            flattened_game[attrib_name] = 1
        else:
            flattened_game[attrib_name] = 0
    flattened_game.pop('steam.categories_description', None)

    # Too many developers are unique. Let's try to simplify it a bit and just count the number of devs.
    flattened_game['steam.developers'] = len(flattened_game['steam.developers'])


    # Replace the 'steam.detailed_description', 'steam.about_the_game', 'steam.short_description' with their sentiment analysis
    # {'neg': 0.11, 'neu': 0.803, 'pos': 0.087, 'compound': -0.8976}
    LONG_FORM_TEXT_ATTRIBS = ['steam.detailed_description', 'steam.about_the_game', 'steam.short_description']
    SENTIMENT_OPTIONS = ['neg', 'neu', 'pos', 'compound']
    for attrib in LONG_FORM_TEXT_ATTRIBS:
        analysis = analyzer.polarity_scores(flattened_game[attrib])
        for sentiment in SENTIMENT_OPTIONS:
            flattened_game[f"{attrib}.{sentiment}"] = analysis[sentiment]
        flattened_game.pop(attrib, None)


    return flattened_game

def get_steam_sale_date_set():
    # We only consider steam sales from https://steamdb.info/sales/history/ that are likely to affect many games
    # These are things like the annual lunar, spring, summer, autumn, halloween, winter sales if they are present (varies slightly by year).
    # This means we leave out end of year awards sales, and niche genre sales (e.g. VR, Polish independence, Golden week) because those sales do not repeat or only affect a small subset of games.
    STEAM_SALE_DATES = [
        # [Y,M,D,Duration(days)]
        [2016,6,23,11],
        [2016,10,28,4],
        [2016,11,23,6],
        [2016,12,22,11],
        [2017,6,22,13],
        [2017,10,26,6],
        [2017,11,22,6],
        [2017,12,21,14],
        [2018,2,15,4],
        [2018,5,24,4],
        [2018,6,21,14],
        [2018,10,29,3],
        [2018,11,21,6],
        [2018,12,20,14],
        [2019,2,4,7],
        [2019,5,24,4],
        [2019,6,25,14],
        [2019,10,28,4],
        [2019,11,26,7],
        [2019,12,19,14],
        [2020,1,23,4],
        [2020,5,21,7],
        [2020,6,25,14],
        [2020,10,29,4],
        [2020,11,25,6],
        [2020,12,22,14],
        [2021,2,11,4],
        [2021,6,24,14],
        [2021,10,28,4],
        [2021,12,22,14],
        [2022,1,27,7],
    ]
    # Use a set for constant r/w speed
    saleDates = set()

    for sale in STEAM_SALE_DATES:
        [y, m, d, duration] = sale
        saleDate = date(y, m, d)
        for i in range(0, duration):
            saleDates.add(saleDate)
            saleDate += timedelta(days=1)

    return saleDates

saleDates = get_steam_sale_date_set()


# Pass in allData here
def find_day_range_and_preprocess(data):
    """
    I want it to be able to detect Data that is:
        - seasonal (steam sales)
        - price decreases after a long time (would this be a trend?)
        - absolute time
        - based off of the game's attributes
    This means, my data should probably have:
        - days since launch column
        - isSteamSale (dates from https://steamdb.info/sales/history/)
        - Year
        - Month
        - Day
        - Add column for % of launch price. This might be more effective at predicting instead of raw number.

    :param data:
    :return:
    """

    daysSet = set()
    timesSet = set()
    timestamps = set()
    for game_obj in data:
        sales = game_obj['itad.sales']
        timestamps.update([row[0] for row in sales])

    for timestamp in timestamps:
        dt_val = datetime.fromtimestamp(int(timestamp) / 1000,tz=timezone.utc)
        timesSet.add(dt_val.time())
        daysSet.add(dt_val.date())

    return

def forward_fill_sales(game):
    sales = game['itad.sales']
    eachDayRecord = []
    [releaseTimestamp, releasePrice] = sales[0]

    # Will never be affected by daylight savings or leap things.
    releaseDt = datetime.fromtimestamp(int(releaseTimestamp) / 1000,tz=timezone.utc).replace(hour=0, minute=30, second=0, microsecond=0)
    workingDay = releaseDt
    # Fill out stuff until the last day's price change
    for sale in sales:
        [saleTimestamp, salePrice] = sale
        saleTimestamp /= 1000
        # some data is bad
        if salePrice == 'NA':
            continue
        while workingDay.timestamp() <= saleTimestamp:
            eachDayRecord.append({
                'year': workingDay.year,
                'month': workingDay.month,
                'day': workingDay.day,
                'daysSinceLaunch': (workingDay - releaseDt).days,
                'price': salePrice,
                'percentOfReleasePrice': salePrice / releasePrice,
                'isSteamSale':  workingDay.date() in saleDates,
            })

            # move 1 day into the future, then set the time to 12:30 AM
            workingDay += timedelta(days=1)
            workingDay = workingDay.replace(hour=0, minute=30, second=0, microsecond=0)
    # Due to the way the data was scraped, there is no need to forward fill.
    return eachDayRecord


cleanData = []
numGames = len(allData)
for ind, game in enumerate(allData):
    if ind % 100 == 0:
        print(f"Game {ind}/{numGames}")
    modified_game = cleanup_data(game)
    modified_game['itad.sales'] = forward_fill_sales(modified_game)
    cleanData.append(modified_game)




Game 0/2227
Game 100/2227
Game 200/2227
Game 300/2227
Game 400/2227
Game 500/2227
Game 600/2227
Game 700/2227
Game 800/2227
Game 900/2227
Game 1000/2227
Game 1100/2227
Game 1200/2227
Game 1300/2227
Game 1400/2227
Game 1500/2227
Game 1600/2227
Game 1700/2227
Game 1800/2227
Game 1900/2227
Game 2000/2227
Game 2100/2227
Game 2200/2227


In [47]:


# We use an off the shelf sentiment analysis since this may play a part in predicting the price, but I kinda doubt it.
# Putting this here in case it actually does.

with open('cleanDataReadable.json', 'w', encoding='utf-8') as f:
    json.dump(cleanData, f, ensure_ascii=False, indent=4)

with open('cleanData.json', 'w', encoding='utf-8') as f:
    json.dump(cleanData, f, ensure_ascii=False)

In [49]:
# Everything has standardized keys. let's list them all.
keys = cleanData[0].keys()
keys

dict_keys(['steamId', 'itadPlain', 'itad.position', 'itad.rank', 'itad.sales', 'steam.name', 'steam.steam_appid', 'steam.required_age', 'steam.is_free', 'steam.dlc', 'steam.reviews', 'steam.pc_requirements', 'steam.mac_requirements', 'steam.linux_requirements', 'steam.developers', 'steam.controller_support', 'steam.metacritic.score', 'steam.drm_consolidated_CrackProof', 'steam.drm_consolidated_Denuvo', 'steam.drm_consolidated_EA', 'steam.drm_consolidated_Games for Windows Live', 'steam.drm_consolidated_GameShield', 'steam.drm_consolidated_Kalypso', 'steam.drm_consolidated_SecuROM', 'steam.drm_consolidated_Ubisoft', 'steam.drm_consolidated_NA', 'steam.supported_languages.english', 'steam.supported_languages.french', 'steam.supported_languages.german', 'steam.supported_languages.dutch', 'steam.supported_languages.czech', 'steam.supported_languages.hungarian', 'steam.supported_languages.italian', 'steam.supported_languages.japanese', 'steam.supported_languages.polish', 'steam.supported_la

In [66]:
import csv

# These will be our columns.
with open('data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = list(cleanData[0].keys())
    fieldnames.remove('itad.sales')
    fieldnames.extend(['steam.ext_user_account_notice', 'year', 'month', 'day', 'daysSinceLaunch', 'price', 'percentOfReleasePrice', 'isSteamSale'])

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for data in cleanData:
        try:
            for day in data['itad.sales']:
                modified_data = copy(data)
                modified_data.pop('itad.sales', None)
                writer.writerow({'steam.ext_user_account_notice': False, 'steam.metacritic.score': -1, **modified_data, **day})
        except:
            print("Error on: ", data)